<a href="https://colab.research.google.com/github/raaz0000002/pytorch_learning/blob/main/Tiles_Creating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-image


In [ ]:
!pip install gdown


In [ ]:
pip install rasterio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 55.5 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window
from tqdm import tqdm

def create_tiles_tif(image_path, output_dir, tile_size=(256, 256), prefix="tile"):
    """
    Splits a TIFF image into smaller tiles and saves them as .tif files.

    Args:
        image_path (str): Path to input .tif image.
        output_dir (str): Directory to save tiles.
        tile_size (tuple): Size of each tile (height, width).
        prefix (str): Prefix for tile filenames.
    """
    os.makedirs(output_dir, exist_ok=True)

    with rasterio.open(image_path) as src:
        width = src.width
        height = src.height
        num_bands = src.count
        profile = src.profile

        tile_height, tile_width = tile_size
        tile_id = 0

        for top in tqdm(range(0, height, tile_height), desc="Creating Tiles"):
            for left in range(0, width, tile_width):
                # Ensure tile doesn't exceed bounds
                w = min(tile_width, width - left)
                h = min(tile_height, height - top)

                window = Window(left, top, w, h)
                transform = src.window_transform(window)
                tile_data = src.read(window=window)

                # Update profile for tile
                tile_profile = profile.copy()
                tile_profile.update({
                    'height': h,
                    'width': w,
                    'transform': transform
                })

                tile_filename = os.path.join(output_dir, f"{prefix}_{tile_id:04d}.tif")
                with rasterio.open(tile_filename, 'w', **tile_profile) as dst:
                    dst.write(tile_data)

                tile_id += 1




In [ ]:
import os
import gdown

# Step 1: Download the TIFF from Google Drive
drive_url = "https://drive.google.com/uc?id=1SqQfWqdnSdXczWaOlcb2OQMzBAuHsDCG"  # convert share link to direct link
local_tif_path = "/content/satellite_image.tif"
gdown.download(drive_url, local_tif_path, quiet=False)

# Step 2: Tile output folder
output_tiles_folder = "/content/drive/MyDrive/Thesis/tiles"

# Step 3: Call the tiling function
create_tiles_tif(local_tif_path, output_tiles_folder)



Downloading...
From (original): https://drive.google.com/uc?id=1SqQfWqdnSdXczWaOlcb2OQMzBAuHsDCG
From (redirected): https://drive.google.com/uc?id=1SqQfWqdnSdXczWaOlcb2OQMzBAuHsDCG&confirm=t&uuid=3c53381d-11e8-475b-b24a-dfad0e8fe034
To: /content/satellite_image.tif
100%|██████████| 953M/953M [00:21<00:00, 44.3MB/s]
Creating Tiles: 100%|██████████| 42/42 [01:32<00:00,  2.21s/it]


In [ ]:
import os
import numpy as np
import rasterio
from skimage.io import imsave
from tqdm import tqdm

def normalize_to_uint8(image, percentile_low=2, percentile_high=98, gamma=1.0):
    """
    Normalize image to 0–255 with percentile clipping and optional gamma correction.

    Args:
        image: Input image as numpy array.
        percentile_low: Lower percentile for clipping (e.g., 2).
        percentile_high: Upper percentile for clipping (e.g., 98).
        gamma: Gamma correction factor (e.g., 0.5 to brighten, >1 to darken).
    """
    image = image.astype(np.float32)

    # Use percentiles to clip outliers
    low_val = np.percentile(image, percentile_low)
    high_val = np.percentile(image, percentile_high)

    # Clip the image to the percentile range
    image = np.clip(image, low_val, high_val)

    # Normalize to 0-1
    image = (image - low_val) / (high_val - low_val)

    # Apply gamma correction (optional)
    if gamma != 1.0:
        image = np.power(image, gamma)

    # Scale to 0-255 and convert to uint8
    image = image * 255
    return image.astype(np.uint8)

def create_tiles_from_tif(tif_path, output_folder, tile_size=(256, 256), stride=(256, 256), gamma=0.5):
    """
    Splits a TIFF image into tiles and saves as PNGs with brightness adjustment.

    Args:
        tif_path (str): Path to local .tif image.
        output_folder (str): Where to save the PNG tiles.
        tile_size (tuple): (Height, Width) of each tile.
        stride (tuple): Stride between tiles (for overlap).
        gamma (float): Gamma correction value (e.g., 0.5 to brighten).
    """
    os.makedirs(output_folder, exist_ok=True)

    try:
        with rasterio.open(tif_path) as src:
            print(f"TIFF dimensions: {src.height}x{src.width}, Bands: {src.count}")

            if src.count >= 3:
                image = src.read([1, 2, 3])  # Read RGB bands
            else:
                image = src.read(1)  # Read single band
                image = np.stack([image] * 3, axis=0)  # Stack to create RGB

            image = np.transpose(image, (1, 2, 0))  # HWC

            # Normalize with percentile clipping and gamma correction
            image = normalize_to_uint8(image, percentile_low=2, percentile_high=98, gamma=gamma)

            if image.max() == 0:
                raise ValueError("Image data is all zeros after normalization")

            H, W, _ = image.shape
            tile_H, tile_W = tile_size

            print(f"Image shape after processing: {image.shape}")

            tile_id = 0
            for i in tqdm(range(0, H - tile_H + 1, stride[0]), desc="Tiling rows"):
                for j in range(0, W - tile_W + 1, stride[1]):
                    tile = image[i:i + tile_H, j:j + tile_W]
                    if tile.max() == 0:
                        print(f"Skipping blank tile at {i},{j}")
                        continue

                    save_path = os.path.join(output_folder, f"tile_{tile_id:04d}.png")
                    imsave(save_path, tile, check_contrast=False)
                    tile_id += 1

            print(f"Generated {tile_id} tiles")

    except Exception as e:
        print(f"Error processing TIFF file: {str(e)}")

# ========== USER INPUT ==========
input_tif_path = "/content/drive/MyDrive/Thesis/22-23-dec_to_feb.tif"  # Your TIFF path
output_folder = "/content/drive/MyDrive/Thesis/Tiles"     # Your output folder

# ========== RUN ==========
create_tiles_from_tif(input_tif_path, output_folder, gamma=1.06)  # Gamma < 1 brightens

TIFF dimensions: 10678x5749, Bands: 16
Image shape after processing: (10678, 5749, 3)


Tiling rows:   0%|          | 0/41 [00:00<?, ?it/s]

Skipping blank tile at 0,0
Skipping blank tile at 0,256
Skipping blank tile at 0,512
Skipping blank tile at 0,768
Skipping blank tile at 0,1024
Skipping blank tile at 0,1280
Skipping blank tile at 0,1536
Skipping blank tile at 0,1792
Skipping blank tile at 0,2048
Skipping blank tile at 0,2304
Skipping blank tile at 0,2560
Skipping blank tile at 0,2816
Skipping blank tile at 0,3072
Skipping blank tile at 0,3328
Skipping blank tile at 0,3584
Skipping blank tile at 0,3840
Skipping blank tile at 0,4096
Skipping blank tile at 0,4352
Skipping blank tile at 0,4608
Skipping blank tile at 0,4864
Skipping blank tile at 0,5120
Skipping blank tile at 0,5376
Skipping blank tile at 256,0
Skipping blank tile at 256,256
Skipping blank tile at 256,512
Skipping blank tile at 256,768
Skipping blank tile at 256,1024
Skipping blank tile at 256,1280
Skipping blank tile at 256,1536
Skipping blank tile at 256,1792
Skipping blank tile at 256,2048
Skipping blank tile at 256,2304
Skipping blank tile at 256,2560


Tiling rows:  10%|▉         | 4/41 [00:00<00:02, 14.82it/s]

Skipping blank tile at 768,4608
Skipping blank tile at 768,4864
Skipping blank tile at 768,5120
Skipping blank tile at 768,5376
Skipping blank tile at 1024,0
Skipping blank tile at 1024,256
Skipping blank tile at 1024,512
Skipping blank tile at 1024,4864
Skipping blank tile at 1024,5120
Skipping blank tile at 1024,5376
Skipping blank tile at 1280,0
Skipping blank tile at 1280,256
Skipping blank tile at 1280,512


...

Skipping blank tile at 9728,2816
Skipping blank tile at 9728,3072
Skipping blank tile at 9728,3328
Skipping blank tile at 9728,3584
Skipping blank tile at 9728,3840
Skipping blank tile at 9728,4096
Skipping blank tile at 9728,4352
Skipping blank tile at 9728,4608
Skipping blank tile at 9728,4864
Skipping blank tile at 9728,5120
Skipping blank tile at 9728,5376


Tiling rows:  98%|█████████▊| 40/41 [00:30<00:00,  1.64it/s]

Skipping blank tile at 9984,2560
Skipping blank tile at 9984,2816
Skipping blank tile at 9984,3072
Skipping blank tile at 9984,3328
Skipping blank tile at 9984,3584
Skipping blank tile at 9984,3840
Skipping blank tile at 9984,4096
Skipping blank tile at 9984,4352
Skipping blank tile at 9984,4608
Skipping blank tile at 9984,4864
Skipping blank tile at 9984,5120
Skipping blank tile at 9984,5376


Tiling rows: 100%|██████████| 41/41 [00:30<00:00,  1.33it/s]

Skipping blank tile at 10240,1792
Skipping blank tile at 10240,2048
Skipping blank tile at 10240,2304
Skipping blank tile at 10240,2560
Skipping blank tile at 10240,2816
Skipping blank tile at 10240,3072
Skipping blank tile at 10240,3328
Skipping blank tile at 10240,3584
Skipping blank tile at 10240,3840
Skipping blank tile at 10240,4096
Skipping blank tile at 10240,4352
Skipping blank tile at 10240,4608
Skipping blank tile at 10240,4864
Skipping blank tile at 10240,5120
Skipping blank tile at 10240,5376
Generated 670 tiles
